In [ ]:
from math import inf

import numpy as np


def line_search_algo(phi, phi_deri=None, phi_ze=None,
                         phi_old=None, phi_i=None,
                         c1=1e-4, c2=0.9, alpha_max=inf):
    """Find alpha that satisfies strong Wolfe conditions.

    alpha > 0 is assumed to be a  descent direction.

    Parameters
    ----------
    phi : callable f(x,*args)
        Objective scalar function.

    derphi : callable f'(x,*args), optional
        Objective function derivative (can be None)
    phi0 : float, optional
        Value of phi at s=0
    old_phi0 : float, optional
        Value of phi at previous point
    derphi0 : float, optional
        Value of derphi at s=0
    args : tuple
        Additional arguments passed to objective function.
    c1 : float
        Parameter for Armijo condition rule.
    c2 : float
        Parameter for curvature condition rule.

    Returns
    -------
    alpha_star : float
        Best alpha
    phi_star
        phi at alpha_star
    phi0
        phi at 0
    derphi_star
        derphi at alpha_star

    the line search algorithm to enforce strong Wolfeconditions.

    """




    if phi_ze is None:
        phi_ze = phi(0.)

    if phi_i is None and phi_deri is not None:
        phi_i = phi_deri(0.)

    alpha0 = 0
    if phi_old is not None:
        alpha1 = min(1.0, 1.01*2*(phi_ze - phi_old)/phi_i)
    else:
        alpha1 = 1.0

    if alpha1 < 0:
        alpha1 = 1.0

    if alpha1 == 0:
        # useless while loop, and raise warnflag=2 due to possible imprecision.
        alpha_new = None
        phi_new = phi_ze
        phi_ze = phi_old
        phi_deri_new = None

    phi_a1 = phi(alpha1)
    #phi_deri_a1 = phi_deri(alpha1)  evaluated below

    phi_a0 = phi_ze
    phi_deri_a0 = phi_i

    i = 1
    maxiter = 10
    for i in range(maxiter):
        if alpha1 == 0:
            break
        if (phi_a1 > phi_ze + c1*alpha1*phi_i) or \
           ((phi_a1 >= phi_a0) and (i > 1)):
            alpha_new, phi_new, phi_deri_new = \
                        zoom_algo(alpha0, alpha1, phi_a0,
                              phi_a1, phi_deri_a0, phi, phi_deri,
                              phi_ze, phi_i, c1, c2)
            break

        phi_deri_a1 = phi_deri(alpha1)
        if (abs(phi_deri_a1) <= -c2*phi_i):
            alpha_new = alpha1
            phi_new = phi_a1
            phi_deri_new = phi_deri_a1
            break

        if (phi_deri_a1 >= 0):
            alpha_new, phi_new, phi_deri_new = \
                        zoom_algo(alpha1, alpha0, phi_a1,
                              phi_a0, phi_deri_a1, phi, phi_deri,
                              phi_ze, phi_i, c1, c2)
            break
        # increase by factor of two on each iteration
        alpha2 = 2 * alpha1
        i = i + 1
        alpha0 = alpha1
        alpha1 = alpha2
        phi_a0 = phi_a1
        phi_a1 = phi(alpha1)
        phi_deri_a0 = phi_deri_a1

    else:
        # stopping test maxiter reached
        alpha_new = alpha1
        phi_new = phi_a1
        phi_deri_new = None

    return alpha_new, phi_new, phi_ze, phi_deri_new

# cubic interlpolation function
def cubic_interpolation (a,fa,fpa,b,fb,c,fc):

    #Finds the minimizer for a cubic polynomial that goes through the
    #points (a,fa), (b,fb), and (c,fc) with derivative at a of fpa.

    #If no minimizer can be found return None

    # f(x) = A *(x-a)^3 + B*(x-a)^2 + C*(x-a) + D

    C = fpa
    D = fa
    db = b-a
    dc = c-a
    if (db == 0) or (dc == 0) or (b==c): return None
    denom = (db*dc)**2 * (db-dc)
    d1 = np.empty((2,2))
    d1[0,0] = dc**2
    d1[0,1] = -db**2
    d1[1,0] = -dc**3
    d1[1,1] = db**3
    [A,B] = np.dot(d1, np.asarray([fb-fa-C*db,fc-fa-C*dc]).flatten())
    A /= denom
    B /= denom
    radical = B*B-3*A*C
    if radical < 0:  return None
    if (A == 0): return None
    xmin = a + (-B + np.sqrt(radical))/(3*A)
    return xmin

#quadratic interpolation function
def quadratic_interpolation(a,fa,fpa,b,fb):

    #Finds the minimizer for a quadratic polynomial that goes through
    #the points (a,fa), (b,fb) with derivative at a of fpa,


    # f(x) = B*(x-a)^2 + C*(x-a) + D
    D = fa
    C = fpa
    db = b-a*1.0
    if (db==0): return None
    B = (fb-D-C*db)/(db*db)
    if (B <= 0): return None
    xmin = a  - C / (2.0*B)
    return xmin

# zoom algorithm according to Optimization book
def zoom_algo(alpha_low, alpha_high, phi_lo, phi_hi, phi_deri_lo,
          phi, phi_deri, phi_ze, phi_i, c1, c2):

    maxiter = 10
    i = 0
    delta1 = 0.2  # cubic interpolant check
    delta2 = 0.1  # quadratic interpolant check
    phi_rec = phi_ze
    a_rec = 0
    while 1:
        # interpolate to find a trial step length between alpha_low and
        # alpha_high Need to choose interpolation here.  Use cubic
        # interpolation and then if the result is within delta *
        # dalpha or outside of the interval bounded by alpha_low or alpha_high
        # then use quadratic interpolation, if the result is still too
        # close, then use bisection

        dalpha = alpha_high-alpha_low;
        if dalpha < 0: a,b = alpha_high,alpha_low
        else: a,b = alpha_low, alpha_high

        # minimizer of cubic interpolant
        # (uses phi_lo, phi_deri_lo, phi_hi, and the most recent value of phi)
        #
        # if the result is too close to the end points (or out of the
        # interval) then use quadratic interpolation with phi_lo,
        # phi_deri_lo and phi_hi if the result is stil too close to the
        # end points (or out of the interval) then use bisection

        if (i > 0):
            cchk = delta1*dalpha
            a_j = cubic_interpolation (alpha_low, phi_lo, phi_deri_lo, alpha_high, phi_hi, a_rec, phi_rec)
        if (i==0) or (a_j is None) or (a_j > b-cchk) or (a_j < a+cchk):
            qchk = delta2*dalpha
            a_j = quadratic_interpolation(alpha_low, phi_lo, phi_deri_lo, alpha_high, phi_hi)
            if (a_j is None) or (a_j > b-qchk) or (a_j < a+qchk):
                a_j = alpha_low + 0.5*dalpha

        # Check new value of a_j

        phi_aj = phi(a_j)
        if (phi_aj > phi_ze + c1*a_j*phi_i) or (phi_aj >= phi_lo):
            phi_rec = phi_hi
            a_rec = alpha_high
            alpha_high = a_j
            phi_hi = phi_aj
        else:
            phi_deri_aj = phi_deri(a_j)
            if abs(phi_deri_aj) <= -c2*phi_i:
                a_star = a_j
                val_star = phi_aj
                valprime_star = phi_deri_aj
                break
            if phi_deri_aj*(alpha_high - alpha_low) >= 0:
                phi_rec = phi_hi
                a_rec = alpha_high
                alpha_high = alpha_low
                phi_hi = phi_lo
            else:
                phi_rec = phi_lo
                a_rec = alpha_low
            alpha_low = a_j
            phi_lo = phi_aj
            phi_deri_lo = phi_deri_aj
        i += 1
        if (i > maxiter):
            a_star = a_j
            val_star = phi_aj
            valprime_star = None
            break
    return a_star, val_star, valprime_star


In [ ]:
import numpy as np
import math
def steepest_descent_line_search(objective, initial_point, alpha, beta, gamma, max_iterations=100, tol=1e-6):
    x = np.array(initial_point)
    history = [x]

    for k in range(max_iterations):
        gradient = approximate_gradient(objective, x)
        norm_gradient = np.linalg.norm(gradient)

        if norm_gradient < tol:
            break

        # Step (a): Choose a descent direction
        descent_direction = -gradient

        # Step (b): Line search to find the step size lambda
        def phi(alpha):
            return objective(x + alpha * descent_direction)

        def phi_deri(alpha):
            return np.dot(approximate_gradient(objective, x + alpha * descent_direction), descent_direction)

        alpha_new, _, _, _ = line_search_algo(phi, phi_deri, c1=alpha, c2=beta)

        # Update x
        x = x + alpha_new * descent_direction
        history.append(x)

    return x, objective(x), history
#gradient algo
def approximate_gradient(objective, x, epsilon=1e-6):
    n = len(x)
    gradient = np.zeros(n)

    for i in range(n):
        perturbed_x1 = x.copy()
        perturbed_x2 = x.copy()
        perturbed_x1[i] += epsilon
        perturbed_x2[i] -= epsilon

        gradient[i] = (objective(perturbed_x1) - objective(perturbed_x2)) / (2 * epsilon)

    return gradient

# Examples
def objective_function1(x):
    return x[0]**4 + x[0]**2 + x[1]**2

initial_points = [(0, 0), (1, 0), (0, 1)]
# alpha,beta,gamma parameter
alpha = 1e-4
beta = 0.9
gamma = 0.1

for initial_point in initial_points:
    result, min_value, history = steepest_descent_line_search(objective_function1, initial_point, alpha, beta, gamma)
    print(f"function 1: Initial Point : {initial_point}, Minimum: {result}, Min Value: {min_value}, Iterations: {len(history)}")

# Example 2
def objective_function2(x):
    return 10 * (x[1] - x[0]**2) + (1 - x[0])**2
def objective_function3(w):
    lam = 1
    result = (
        0.5 * lam * (w[0] * w[0] + w[1] * w[1] + w[2] * w[2]) +
        np.log(1 + np.exp(-(w[0] + w[1] * 2.0 + w[2]))) +
        np.log(1 + np.exp(-(w[0] * 2.0 + w[1] * 4.0 + w[2]))) +
        np.log(1 + np.exp(-(w[0] * 2.5 + w[1] * 3.5 + w[2]))) +
        np.log(1 + np.exp(-(w[0] * 3.0 + w[1] * 1.5 + w[2]))) +
        np.log(1 + np.exp(-(w[0] * 3.5 + w[1] * 4.5 + w[2]))) +
        np.log(1 + np.exp((w[0] * 4 + w[1] * 1.5 + w[2]))) +
        np.log(1 + np.exp((w[0] * 4 + w[1] * 3.5 + w[2]))) +
        np.log(1 + np.exp((w[0] * 4.5 + w[1] * 2.5 + w[2]))) +
        np.log(1 + np.exp((w[0] * 5 + w[1] + w[2]))) +
        np.log(1 + np.exp((w[0] * 6 + w[1] * 2 + w[2]))) +
        np.log(1 + np.exp(-(w[0] * 6.5 + w[1] * 3 + w[2]))) +
        np.log(1 + np.exp((w[0] * 7 + w[1] * 1.5 + w[2])))
    )
    return result
    return result


initial_points = [(0, 0), (1, 0), (0, 1)]

for initial_point in initial_points:
    result, min_value, history = steepest_descent_line_search(objective_function2, initial_point, alpha, beta, gamma)
    print(f"function 2: Initial Point: {initial_point}, Minimum: {result}, Min Value: {min_value}, Iterations: {len(history)}")
initial_points1 = [(0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1)]

for initial_point in initial_points1:
    result, min_value, history = steepest_descent_line_search(objective_function3, initial_point, alpha, beta, gamma)
    print(f"Objective 3: Initial Point: {initial_point}, Minimum: {result}, Min Value: {min_value}, Iterations: {len(history)}")

function 1: Initial Point : (0, 0), Minimum: [0 0], Min Value: 0, Iterations: 1
function 1: Initial Point : (1, 0), Minimum: [3.33603769e+21 0.00000000e+00], Min Value: 1.2385792282542147e+86, Iterations: 5
function 1: Initial Point : (0, 1), Minimum: [0.00000000e+00 2.84217094e-14], Min Value: 8.077935669463161e-28, Iterations: 3
function 2: Initial Point: (0, 0), Minimum: [0 0], Min Value: 1, Iterations: 1
function 2: Initial Point: (1, 0), Minimum: [1.36704e+11 0.00000e+00], Min Value: -1.6819185254673408e+23, Iterations: 3
function 2: Initial Point: (0, 1), Minimum: [ 2.08080078e+10 -2.56001562e+09], Min Value: -3.8967587021927654e+21, Iterations: 5
Objective 3: Initial Point: (0, 0, 0), Minimum: [0 0 0], Min Value: 8.317766166719343, Iterations: 1


<ipython-input-49-f06a7d47c05e>:67: RuntimeWarning: overflow encountered in exp
  np.log(1 + np.exp(-(w[0] + w[1] * 2.0 + w[2]))) +
<ipython-input-49-f06a7d47c05e>:68: RuntimeWarning: overflow encountered in exp
  np.log(1 + np.exp(-(w[0] * 2.0 + w[1] * 4.0 + w[2]))) +
<ipython-input-49-f06a7d47c05e>:69: RuntimeWarning: overflow encountered in exp
  np.log(1 + np.exp(-(w[0] * 2.5 + w[1] * 3.5 + w[2]))) +
<ipython-input-49-f06a7d47c05e>:70: RuntimeWarning: overflow encountered in exp
  np.log(1 + np.exp(-(w[0] * 3.0 + w[1] * 1.5 + w[2]))) +
<ipython-input-49-f06a7d47c05e>:71: RuntimeWarning: overflow encountered in exp
  np.log(1 + np.exp(-(w[0] * 3.5 + w[1] * 4.5 + w[2]))) +
<ipython-input-49-f06a7d47c05e>:77: RuntimeWarning: overflow encountered in exp
  np.log(1 + np.exp(-(w[0] * 6.5 + w[1] * 3 + w[2]))) +


Objective 3: Initial Point: (1, 0, 0), Minimum: [nan nan nan], Min Value: nan, Iterations: 101
Objective 3: Initial Point: (0, 1, 0), Minimum: [nan nan nan], Min Value: nan, Iterations: 101
Objective 3: Initial Point: (0, 0, 1), Minimum: [nan nan nan], Min Value: nan, Iterations: 101


reference "https://github.com/minrk/scipy-1/blob/master/scipy/optimize/linesearch.py"
